# Tutorial 1. Unperturbed Earth-orbiting Satellite

## Import statements

In [1]:
import numpy as np
import pandas as pd
from tudatpy import elements
from tudatpy.kernel import constants
from tudatpy.kernel import numerical_integrators
from tudatpy.kernel import spice_interface
from tudatpy.kernel import propagators
from tudatpy.kernel import simulation_setup

## Define tutorial variables

In [2]:
start_epoch=0.0
fixed_step_size=10.0
end_epoch=constants.JULIAN_DAY
parent_body="Earth"
frame_orientation="ECLIPJ2000"
frame_origin="SSB"
satellite_name="Delfi-C3"
sat_sma=7500.0E3
sat_ecc=0.1
sat_inc=np.deg2rad(85.3)
sat_raan=np.deg2rad(23.4)
sat_argp=np.deg2rad(235.7)
sat_nu=np.deg2rad(139.87)
return_output=False
print_output=True
output_type='markdown'

## Create environment

In [3]:
spice_interface.load_standard_kernels()

# Create body objects.
bodies_to_create = [parent_body]

body_settings = simulation_setup.get_default_body_settings(bodies_to_create)

body_settings[parent_body].ephemeris_settings = simulation_setup.ConstantEphemerisSettings(
    np.zeros(6))

body_settings[parent_body].rotation_model_settings.reset_original_frame(frame_orientation)

# Create Earth Object.
bodies = simulation_setup.create_bodies(body_settings)

## Create vehicle

In [4]:
bodies[satellite_name] = simulation_setup.Body()

## Finalise bodies

In [5]:
simulation_setup.set_global_frame_body_ephemerides(bodies, "SSB", "ECLIPJ2000")

## Define accelerations

In [6]:
# Define bodies that are propagated.
bodies_to_propagate = [satellite_name]

# Define central bodies.
central_bodies = [parent_body]

# Define accelerations acting on Delfi-C3.
accelerations_of_delfi_c3 = dict(
    Earth=[simulation_setup.Acceleration.point_mass_gravity()]
)

# Create global accelerations dictionary.
accelerations = {satellite_name: accelerations_of_delfi_c3}

# Create acceleration models.
acceleration_models = simulation_setup.create_acceleration_models_dict(
    bodies, accelerations, bodies_to_propagate, central_bodies)

## Define propagation settings

In [7]:
# Set initial conditions for the Asterix satellite that will be
# propagated in this simulation. The initial conditions are given in
# Keplerian elements and later on converted to Cartesian elements.

# Set Keplerian elements for Delfi-C3
earth_gravitational_parameter = bodies[
    parent_body].gravity_field_model.get_gravitational_parameter()

# REVISED CONTEMPORARY DESIGN.
system_initial_state = elements.keplerian2cartesian(
    mu=earth_gravitational_parameter,
    a=sat_sma,
    ecc=sat_ecc,
    inc=sat_inc,
    raan=sat_raan,
    argp=sat_argp,
    nu=sat_nu)

# Create propagation settings.
propagator_settings = propagators.TranslationalStatePropagatorSettings(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    system_initial_state,
    end_epoch)

# Create numerical integrator settings.
integrator_settings = numerical_integrators.IntegratorSettings(
    numerical_integrators.AvailableIntegrators.rungeKutta4,
    end_epoch,
    fixed_step_size)

## Propagate orbit

In [8]:
###########################################################################
# PROPAGATE ORBIT #########################################################
###########################################################################

# Create simulation object and propagate dynamics.
dynamics_simulator = propagators.SingleArcDynamicsSimulator(
    bodies, integrator_settings, propagator_settings, True)
result = dynamics_simulator.get_equations_of_motion_numerical_solution()

## Print output

In [9]:

print(
    f"""
Single Earth-Orbiting Satellite Example.
The initial position vector of Delfi-C3 is [km]: \n{
    result[list(result.keys())[0]][:3] / 1E3} 
The initial velocity vector of Delfi-C3 is [km]: \n{
    result[list(result.keys())[0]][3:] / 1E3}
After {end_epoch} seconds the position vector of Delfi-C3 is [km]: \n{
    result[list(result.keys())[-1]][:3] / 1E3}
And the velocity vector of Delfi-C3 is [km]: \n{
    result[list(result.keys())[-1]][3:] / 1E3}
    """
)


Single Earth-Orbiting Satellite Example.
The initial position vector of Delfi-C3 is [km]: 
[7037.48400133 3238.05901792 2150.7241875 ] 
The initial velocity vector of Delfi-C3 is [km]: 
[-1.46565763 -0.04095839  6.62279761]
After 86400.0 seconds the position vector of Delfi-C3 is [km]: 
[7022.55787059 3237.52532674 2216.86888198]
And the velocity vector of Delfi-C3 is [km]: 
[-1.51953372 -0.06577191  6.60606169]
    
